## 前置：垃圾分类玩法.ipynb，完成`calibration_ok`

# 说明
这个笔记本用于Atlas 200DK摄像头采集图片数据，保存到Atlas 200DK本地文件夹。
启动后，用户可以实时查看200DK摄像头视频画面，每点击一次**保存图片**按钮，会将当前视频画面截获，并保存到本地文件夹

- 文件夹命令：Images-<系统时间> , 例如：Images-2022-11-10-13_36_54
- 图片命名：<系统时间>.jpg , 例如：2022-11-10-13_43_26.jpg

# 依赖条件
1. 下载[Ascend Sample代码仓](https://gitee.com/ascend/samples)到A200DK
2. 参考[Ascend Sample代码仓 python环境准备](https://gitee.com/ascend/samples/tree/master/python/environment)章节准备依赖环境，主要是`acllite`库的安装和环境变量设置


# 实现说明
通过4个线程（摄像头实时画面捕获线程、界面显示线程、图片文件保存线程、主线程）来完成图片数据的采集、保存。主线程用于整个程序的启动-停止控制。
其中用到了`queue.Queue`模块来缓存图片数据，使用缓存队列的目的是对实时摄像头画面截获 与 慢速图片数据写入文件 2个线程之间传递数据。


In [1]:
import os
import sys
import time
import queue
import threading

import cv2
import numpy as np
import ipywidgets as widgets
from ipyevents import Event 
from IPython.display import display
from dofbot_config import *

pkg_dir = os.path.dirname(os.path.realpath("__file__"))
cfg_dir = os.path.join(pkg_dir, "config")
XYT_cfg_dir = os.path.join(cfg_dir, "XYT_config.txt")
data_idx_cfg_dir = os.path.join(cfg_dir, "data_collect_idx.txt")
dp_cfg_dir = os.path.join(cfg_dir, "dp.bin")
XYT_path = XYT_cfg_dir


CAMERA_FRAME_WIDTH = 640
CAMERA_FRAME_HEIGHT = 480

# https://pypi.org/project/ipywidgets/
imgbox = widgets.Image(format='jpg', height=CAMERA_FRAME_HEIGHT, width=CAMERA_FRAME_WIDTH)


FLAG = True
IMAGE = False
CALIB = True # REVISE

q = queue.Queue(maxsize=10)
calibration = Arm_Calibration()

uuid_str = time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime())
tmp_folder_name ='Images-%s' % uuid_str
os.makedirs(tmp_folder_name, exist_ok=True)


# 获取摄像头画面并实时显示 线程函数
def capture_video():
    display(imgbox)
    
    capture = cv2.VideoCapture(0)
    
    while capture.isOpened():
        # Read a picture from the camera
        _, image = capture.read()
        if image is None:
            print("Get memory from camera failed")
            break
        
        if CALIB:
            dp = np.fromfile(dp_cfg_dir, dtype=np.int32)
            dp = dp.reshape(4,2)

            image = calibration.Perspective_transform(dp, image)
            
        imgbox.value = cv2.imencode('.jpg', image)[1].tobytes()
        
        global IMAGE
        if IMAGE:
            if q.full():
                continue
            else:
                q.put(image)
            IMAGE = False


# 捕获摄像头画面 IMAGE 控制全局变量
def cap_image(change):
    global IMAGE
    IMAGE = True

# 捕获摄像头画面按钮显示 线程函数
def button_control():
    # 创建按钮
    button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
    get_image_button = widgets.Button(description='保存图片', button_style='danger', layout=button_layout)
    
    # 显示按钮
    middle_box = widgets.HBox([get_image_button], layout=widgets.Layout(align_self='center'))
    
    # link buttons to actions
    get_image_button.on_click(cap_image) 
    display(middle_box)


# 保存图片到文件 线程函数
def save_image():
    while FLAG:
        img = q.get()
        with open(data_idx_cfg_dir, "r") as f:
            contents = f.readline()
            count = int(contents)
        uuid_str = str(count)
        tmp_file_name ='%s.jpg' % uuid_str
        print(os.path.join(tmp_folder_name, tmp_file_name))
        print(img.shape)
        print(type(img))
        if not cv.imwrite(os.path.join(tmp_folder_name, tmp_file_name), img):
            raise Exception("Could not write images")
        
        with open(data_idx_cfg_dir, "w") as f:
            count += 1
            f.write(str(count))
    

In [2]:
thread1 = threading.Thread(name='t1',target=capture_video, args=())
thread1.start()

thread2 = threading.Thread(name='t2',target=button_control, args=())
thread2.start()

thread3 = threading.Thread(name='t3',target=save_image, args=())
thread3.start()

Image(value=b'', format='jpg', height='480', width='640')

In [ ]:
FLAG = False
thread1.join()
thread2.join()
thread3.join()